In [1]:
import os
import zarr
import timm
import random
import json
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict
import sys
import torch

# import torchvision.transforms.functional as F
import random

warnings.filterwarnings("ignore")
sys.path.append("./src/")

from src.config import CFG
from src.dataloader import (
    read_zarr,
    read_info_json,
    scale_coordinates,
    create_dataset,
    create_segmentation_map,
    EziiDataset,
    drop_padding,
)
from src.network import Unet3D
from src.utils import save_images, PadToSize
from src.metric import (
    score,
    create_cls_pos,
    create_cls_pos_sikii,
    create_df,
    SegmentationLoss,
    DiceLoss,
)
from metric import visualize_epoch_results
from src.utils import save_images
from src.metric import score, create_cls_pos, create_cls_pos_sikii, create_df
from src.inference import inference, inference2pos
from src.kaggle_notebook_metric import compute_lb

sample_submission = pd.read_csv("../../inputs/sample_submission.csv")

In [2]:
def create_gt_df(base_dir, exp_names):
    result_df = None
    particle_names = CFG.particles_name

    for exp_name in exp_names:
        for particle in particle_names:
            np_corrds = read_info_json(
                base_dir=base_dir, exp_name=exp_name, particle_name=particle
            )  # (n, 3)
            # 各行にexp_nameとparticle_name追加
            particle_df = pd.DataFrame(np_corrds, columns=["z", "y", "x"])
            particle_df["experiment"] = exp_name
            particle_df["particle_type"] = particle

            if result_df is None:
                result_df = particle_df
            else:
                result_df = pd.concat([result_df, particle_df], axis=0).reset_index(
                    drop=True
                )

    result_df = result_df.reset_index()
    result_df = result_df[["index", "experiment", "particle_type", "x", "y", "z"]]

    return result_df


gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", CFG.train_exp_names)
gt_df

,index,experiment,particle_type,x,y,z
0,0,TS_73_6,apo-ferritin,268.662,4730.318,916.115
1,1,TS_73_6,apo-ferritin,238.946,4853.061,909.898
2,2,TS_73_6,apo-ferritin,83.114,5729.560,1219.524
3,3,TS_73_6,apo-ferritin,582.143,2769.968,1076.364
4,4,TS_73_6,apo-ferritin,510.389,2157.244,362.438
...,...,...,...,...,...,...
1124,1124,TS_6_6,virus-like-particle,2609.876,4569.876,1169.759
1125,1125,TS_6_6,virus-like-particle,2213.287,4135.017,1286.851
1126,1126,TS_6_6,virus-like-particle,3303.905,5697.825,789.744
1127,1127,TS_6_6,virus-like-particle,1008.748,5949.213,1077.303


In [3]:
import timm

encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder, num_domains=5).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

# inferenced_array = inference(model, exp_name, train=False)
# 0.7303962244998289

<All keys matched successfully>

In [4]:
exp_names = CFG.valid_exp_names  # ["TS_6_4", "TS_5_4", "TS_69_2"]

import gc
from tqdm import tqdm


constant = 0.25
sikii = {
    "apo-ferritin": constant,
    "beta-amylase": constant,
    "beta-galactosidase": constant,
    "ribosome": constant,
    "thyroglobulin": constant,
    "virus-like-particle": constant,
}

pred_dict = {}

# for exp_name in tqdm(CFG.train_exp_names):
for exp_name in tqdm(exp_names):  # 5つのデータで試す
    # inferenced_array = inference(model, exp_name, train=False)
    inferenced_array, n_tomogram, segmentation_map = inference(
        model,
        exp_name,
        train=False,
        base_dir="../../inputs/train/",
    )
    pred_dict[exp_name] = inferenced_array
    # pred_df = inference2pos(
    #     pred_segmask=inferenced_array, exp_name=exp_name, sikii_dict=sikii
    # )

    # all_pred.append(pred_df)

    gc.collect()

100%|██████████| 1/1 [00:15<00:00, 15.49s/it]


In [5]:
# pred_df = pd.concat(all_pred, axis=0).reset_index(drop=True)
# pred_df = pred_df[pred_df["particle_type"] != "beta-amylase"]
# pred_df = pred_df.drop_duplicates(
#     subset=["experiment", "x", "y", "z"], keep="first"
# ).reset_index(drop=True)
# pred_df = pred_df.reset_index().rename(columns={"index": "id"})
# pred_df

In [6]:
gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", exp_names)
gt_df

,index,experiment,particle_type,x,y,z
0,0,TS_5_4,apo-ferritin,468.514,5915.906,604.167
1,1,TS_5_4,apo-ferritin,5674.694,1114.354,565.068
2,2,TS_5_4,apo-ferritin,5744.509,1049.172,653.712
3,3,TS_5_4,apo-ferritin,5880.769,1125.348,579.560
4,4,TS_5_4,apo-ferritin,4661.667,1269.497,810.409
...,...,...,...,...,...,...
135,135,TS_5_4,virus-like-particle,2636.539,4214.980,965.410
136,136,TS_5_4,virus-like-particle,3137.396,3572.460,372.914
137,137,TS_5_4,virus-like-particle,3294.133,3027.464,674.070
138,138,TS_5_4,virus-like-particle,2997.686,4948.218,1169.375


In [7]:
# sikii値とexp_namesを入れるとスコアを出力する関数


def compute_score(sikii_list, inferenced_array, exp_name):
    apo_ferritin = sikii_list[0]
    beta_amylase = sikii_list[1]
    beta_galactosidase = sikii_list[2]
    ribosome = sikii_list[3]
    thyroglobulin = sikii_list[4]
    virus_like_particle = sikii_list[5]

    sikii_dict = {
        "apo-ferritin": apo_ferritin,
        "beta-amylase": beta_amylase,
        "beta-galactosidase": beta_galactosidase,
        "ribosome": ribosome,
        "thyroglobulin": thyroglobulin,
        "virus-like-particle": virus_like_particle,
    }

    all_pred = []

    pred_df = inference2pos(
        pred_segmask=inferenced_array, exp_name=exp_name, sikii_dict=sikii_dict
    )

    all_pred.append(pred_df)

    gc.collect()

    pred_df = pd.concat(all_pred, axis=0).reset_index(drop=True)
    pred_df = pred_df[pred_df["particle_type"] != "beta-amylase"]
    pred_df = pred_df.drop_duplicates(
        subset=["experiment", "x", "y", "z"], keep="first"
    ).reset_index(drop=True)
    pred_df = pred_df.reset_index().rename(columns={"index": "id"})

    gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", [exp_name])

    result_df, lb_score = compute_lb(
        pred_df, "../../inputs/train/overlay/ExperimentRuns/", [exp_name]
    )

    return lb_score


def reduce_computation_sikii_search(
    inferenced_array: np.ndarray, exp_name: str, threshold_candidates: list[float]
) -> tuple[list[float], float]:
    """
    # How
    6つのしきい値が互いに独立してスコアに貢献しているという前提で、
    1次元ずつ最適なしきい値を探す手法を実装する.

    1. 初期の best_thresholds (全要素 0.5 など適当な値) を用意
    2. i=0 から i=5 まで順番に:
       - threshold_candidates をすべて試し、他は固定したまま i 番目だけ変化させてスコアを計算
       - 最良スコアが得られる候補値を確定し、best_thresholds[i] とする
    3. 全部決まったら最終的なスコアを計算して返す

    これにより、全組み合わせ (product) を回すよりも計算量が大幅に減少する.
    """
    # Why not: 6値独立であるという前提が満たされていない場合、近似解になる可能性あり
    best_thresholds = [0.5] * 6  # 適当な初期値でOK

    for i in tqdm(range(6)):
        best_local_score = -float("inf")
        best_local_value = None

        for candidate in threshold_candidates:
            current_thresholds = best_thresholds[:]  # 現在のベストを複製
            current_thresholds[i] = candidate
            score = compute_score(current_thresholds, inferenced_array, exp_name)
            if score > best_local_score:
                best_local_score = score
                best_local_value = candidate

        # i番目のしきい値を最適値に更新
        best_thresholds[i] = best_local_value

    final_score = compute_score(best_thresholds, inferenced_array, exp_name)
    return best_thresholds, final_score

In [8]:
# compute_score([0.25, 0.25, 0.25, 0.25, 0.25, 0.25], exp_names)

In [9]:
# 平均値
from sklearn.metrics import *
from scipy.optimize import minimize
import gc
import numpy as np
import pandas as pd
from scipy.optimize import differential_evolution

exp_name = CFG.valid_exp_names[0]

inferenced_array, n_tomogram, segmentation_map = inference(
    model,
    exp_name,
    train=False,
    base_dir="../../inputs/train/",
)

In [10]:
# KappaOPtimizer = minimize(
#     compute_score,
#     x0=[0.90, 0.90, 0.90, 0.90, 0.90, 0.90],
#     args=(inferenced_array, exp_name),
#     bounds=[(0.0, 0.95) for _ in range(6)],
#     method="nelder-mead",
#     options={"maxiter": 50},
# )

best_thresholds, final_score = reduce_computation_sikii_search(
    inferenced_array,
    exp_name,
    [
        0.05,
        0.1,
        0.15,
        0.2,
        0.25,
        0.3,
        0.35,
        0.4,
        0.45,
        0.5,
        0.55,
        0.6,
        0.65,
        0.7,
        0.75,
        # 0.8,
        # 0.85,
        # 0.9,
        # 0.95,
    ],
)

100%|██████████| 6/6 [03:33<00:00, 35.58s/it]


In [11]:
compute_score(best_thresholds, inferenced_array, exp_name)

0.7043184659376586

In [12]:
best_thresholds

[0.75, 0.05, 0.5, 0.6, 0.25, 0.5]

In [13]:
apo_ferritin = best_thresholds[0]
beta_amylase = best_thresholds[1]
beta_galactosidase = best_thresholds[2]
ribosome = best_thresholds[3]
thyroglobulin = best_thresholds[4]
virus_like_particle = best_thresholds[5]

sikii_dict = {
    "apo-ferritin": apo_ferritin,
    "beta-amylase": beta_amylase,
    "beta-galactosidase": beta_galactosidase,
    "ribosome": ribosome,
    "thyroglobulin": thyroglobulin,
    "virus-like-particle": virus_like_particle,
}

sikii_dict

{'apo-ferritin': 0.75,
 'beta-amylase': 0.05,
 'beta-galactosidase': 0.5,
 'ribosome': 0.6,
 'thyroglobulin': 0.25,
 'virus-like-particle': 0.5}